In [ ]:
import alpaca_trade_api as tradeapi
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
# Define your keys
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKSZXS4Q79O1VIXILWAS"
ALPACA_SECRET_KEY = "Ekki5szEyl4MtqHZzh788ZF8kIi43DEEmxmT8k6S"
# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, base_url=BASE_URL)
# Define the date range for backtesting (last 4 months)
end_date = datetime.now()
start_date = end_date - timedelta(days=4 * 30)
# Fetch daily data for Apple stock
symbol = 'AAPL'
stock_data = yf.download(symbol, start=start_date, end=end_date, interval='1h')
# Existing code for calculating pivot points, support/resistance, etc.
pivot_point = (stock_data['High'] + stock_data['Low'] + stock_data['Close']) / 3
# Calculate support and resistance levels
first_support = (2 * pivot_point) - stock_data['High']
first_resistance = (2 * pivot_point) - stock_data['Low']
second_support = pivot_point - (stock_data['High'] - stock_data['Low'])
second_resistance = pivot_point + (stock_data['High'] - stock_data['Low'])
third_support = pivot_point - 2 * (stock_data['High'] - stock_data['Low'])
third_resistance = pivot_point + 2 * (stock_data['High'] - stock_data['Low'])
# Create a new DataFrame to store the pivot points and support/resistance levels
pivot_df = pd.DataFrame(index=stock_data.index)
pivot_df['Pivot'] = pivot_point
pivot_df['Support 1'] = first_support
pivot_df['Resistance 1'] = first_resistance
pivot_df['Support 2'] = second_support
pivot_df['Resistance 2'] = second_resistance
pivot_df['Support 3'] = third_support
pivot_df['Resistance 3'] = third_resistance
# Merge the pivot DataFrame with the stock data DataFrame
stock_data_with_pivot = pd.concat([stock_data, pivot_df], axis=1)
# Create signals based on pivot points
stock_data_with_pivot['Signal'] = 0
stock_data_with_pivot.loc[stock_data_with_pivot['Low'] < stock_data_with_pivot['Support 1'], 'Signal'] = 1
stock_data_with_pivot.loc[stock_data_with_pivot['Low'] < stock_data_with_pivot['Support 2'], 'Signal'] = 2
stock_data_with_pivot.loc[stock_data_with_pivot['Low'] < stock_data_with_pivot['Support 3'], 'Signal'] = 3
stock_data_with_pivot.loc[stock_data_with_pivot['High'] > stock_data_with_pivot['Resistance 1'], 'Signal'] = -1
stock_data_with_pivot.loc[stock_data_with_pivot['High'] > stock_data_with_pivot['Resistance 2'], 'Signal'] = -2
stock_data_with_pivot.loc[stock_data_with_pivot['High'] > stock_data_with_pivot['Resistance 3'], 'Signal'] = -3
# Generate trading orders based on the signals
backtest_results = pd.DataFrame(index=stock_data_with_pivot.index, columns=['Order', 'Price', 'Quantity'])
for index, row in stock_data_with_pivot.iterrows():
    if row['Signal'] > 0:
        backtest_results.loc[index, 'Order'] = 'Buy'
        backtest_results.loc[index, 'Price'] = stock_data_with_pivot.loc[index, 'Close']
        backtest_results.loc[index, 'Quantity'] = 100
    elif row['Signal'] < 0:
        backtest_results.loc[index, 'Order'] = 'Sell'
        backtest_results.loc[index, 'Price'] = stock_data_with_pivot.loc[index, 'Close']
        backtest_results.loc[index, 'Quantity'] = 100
# Calculate the portfolio value
backtest_results['Portfolio Value'] = 0
cash = 100000  # Initial capital
shares = 0
for index, row in backtest_results.iterrows():
    if row['Order'] == 'Buy':
        shares += row['Quantity']
        cash -= row['Price'] * row['Quantity']
    elif row['Order'] == 'Sell':
        shares -= row['Quantity']
        cash += row['Price'] * row['Quantity']
    backtest_results.loc[index, 'Portfolio Value'] = cash + (shares * row['Price'])
# Print the backtest results
pd.set_option('display.max_rows', None)
backtest_results